In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from supabase import create_client, Client
from tqdm import tqdm
import json
import time
import logging
import os
from dotenv import load_dotenv
from rich.console import Console
from rich.progress import Progress, BarColumn, TextColumn, TimeRemainingColumn
from datetime import datetime
import re

In [28]:
# Load variables from .env into the environment
load_dotenv()

# Read variables
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

In [29]:
# Initialize client
supabase: Client = create_client(supabase_url, supabase_key)

In [30]:
table_name = "autoscout_car_adverts"

In [31]:
response = supabase.table(table_name).select("*").limit(1).execute()

In [32]:
response

APIResponse[~_ReturnT](data=[{'id': 77546, 'car_id': '68936f70-81e9-4387-ad5c-ac6eed1f5438', 'scraped_at': '2025-08-12T12:59:52.134215', 'make': 'nissan', 'model': 'qashqai', 'fuel_type': '2', 'first_registration': '03-2023', 'mileage': 48072, 'post_code_raw': '8281 PC', 'listing_price': 28695, 'post_code': '8281PC', 'transmission': None, 'fuel_text': None, 'power_text': None, 'power_kw': None, 'power_pk': None, 'model_type': None, 'model_text': None, 'range_raw': None, 'range_general': None, 'range_urban': None}], count=None)

In [33]:
response.data

[{'id': 77546,
  'car_id': '68936f70-81e9-4387-ad5c-ac6eed1f5438',
  'scraped_at': '2025-08-12T12:59:52.134215',
  'make': 'nissan',
  'model': 'qashqai',
  'fuel_type': '2',
  'first_registration': '03-2023',
  'mileage': 48072,
  'post_code_raw': '8281 PC',
  'listing_price': 28695,
  'post_code': '8281PC',
  'transmission': None,
  'fuel_text': None,
  'power_text': None,
  'power_kw': None,
  'power_pk': None,
  'model_type': None,
  'model_text': None,
  'range_raw': None,
  'range_general': None,
  'range_urban': None}]

## Scraping

In [34]:
base_url = "https://www.autoscout24.nl/lst"

In [35]:
params = {
    "atype": "C",
    "cy": "NL",
    "damaged_listing": "exclude",
    "desc": "1",
    "powertype": "kw",
    "sort": "age",
    "source": "homepage_search-mask",
    "ustate": "N,U",
    "kmfrom":0,
    "kmto":1000,
    "pricefrom":0,
    "priceto":10000,
    "page": 1  # start page
}

In [36]:
price_vec = np.array(
        [0, 500, 650, 700, 750, 850, 1000, 1100, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 4000, 4500,
         5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000,
         13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500, 18000, 18500, 19000, 19500, 20000, 20500, 21000,
         21500, 22000, 22500, 23000, 24000, 24500, 25000, 26000, 27000, 28000, 28500, 29000, 30000, 31000, 32000, 33000,
         34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000,
         50000, 52000, 54000, 56000, 58000, 60000, 62000, 64000, 66000, 68000, 70000, 75000, 80000, 85000, 90000, 95000,
         100000, 150000, 1e9])
km_vec = np.array(
        [0, 1, 2, 5, 10, 15, 20, 50, 100, 200, 500, 1000, 2000, 3000, 5000, 10000, 15000, 20000, 25000, 30000, 35000,
         40000, 45000, 50000, 55000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 145000, 150000,
         155000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 260000, 280000, 300000, 350000,
         400000, 1e9])

In [37]:
price_vec

array([0.00e+00, 5.00e+02, 6.50e+02, 7.00e+02, 7.50e+02, 8.50e+02,
       1.00e+03, 1.10e+03, 1.25e+03, 1.50e+03, 1.75e+03, 2.00e+03,
       2.25e+03, 2.50e+03, 2.75e+03, 3.00e+03, 3.25e+03, 3.50e+03,
       4.00e+03, 4.50e+03, 5.00e+03, 5.50e+03, 6.00e+03, 6.50e+03,
       7.00e+03, 7.50e+03, 8.00e+03, 8.50e+03, 9.00e+03, 9.50e+03,
       1.00e+04, 1.05e+04, 1.10e+04, 1.15e+04, 1.20e+04, 1.25e+04,
       1.30e+04, 1.35e+04, 1.40e+04, 1.45e+04, 1.50e+04, 1.55e+04,
       1.60e+04, 1.65e+04, 1.70e+04, 1.75e+04, 1.80e+04, 1.85e+04,
       1.90e+04, 1.95e+04, 2.00e+04, 2.05e+04, 2.10e+04, 2.15e+04,
       2.20e+04, 2.25e+04, 2.30e+04, 2.40e+04, 2.45e+04, 2.50e+04,
       2.60e+04, 2.70e+04, 2.80e+04, 2.85e+04, 2.90e+04, 3.00e+04,
       3.10e+04, 3.20e+04, 3.30e+04, 3.40e+04, 3.50e+04, 3.60e+04,
       3.70e+04, 3.80e+04, 3.90e+04, 4.00e+04, 4.10e+04, 4.20e+04,
       4.30e+04, 4.40e+04, 4.50e+04, 4.60e+04, 4.70e+04, 4.80e+04,
       4.90e+04, 5.00e+04, 5.20e+04, 5.40e+04, 5.60e+04, 5.80e

In [38]:
km_vec

array([0.00e+00, 1.00e+00, 2.00e+00, 5.00e+00, 1.00e+01, 1.50e+01,
       2.00e+01, 5.00e+01, 1.00e+02, 2.00e+02, 5.00e+02, 1.00e+03,
       2.00e+03, 3.00e+03, 5.00e+03, 1.00e+04, 1.50e+04, 2.00e+04,
       2.50e+04, 3.00e+04, 3.50e+04, 4.00e+04, 4.50e+04, 5.00e+04,
       5.50e+04, 6.00e+04, 7.00e+04, 8.00e+04, 9.00e+04, 1.00e+05,
       1.10e+05, 1.20e+05, 1.30e+05, 1.40e+05, 1.45e+05, 1.50e+05,
       1.55e+05, 1.60e+05, 1.70e+05, 1.80e+05, 1.90e+05, 2.00e+05,
       2.10e+05, 2.20e+05, 2.30e+05, 2.40e+05, 2.60e+05, 2.80e+05,
       3.00e+05, 3.50e+05, 4.00e+05, 1.00e+09])

In [83]:
count_added = 0
cars_to_insert = []
car_ids_in_upsert = set()
batch_size = 500
page_limit_autoscout = 20
refresh_rate_cars_in_database = 10

print("Fetching existing car IDs from the database...")
response = supabase.table(table_name).select("car_id").execute()
car_ids_in_database = {d['car_id'] for d in response.data}
print(f"Found {len(car_ids_in_database)} existing car IDs.")

Fetching existing car IDs from the database...
Found 268576 existing car IDs.


In [ ]:
# Get current date and time
now = datetime.now()

# Format it as YYYY-MM-DD_HH-MM-SS
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# The level is set to INFO, so all messages from INFO and above will be recorded.
logging.basicConfig(filename=f"../logging/script_log_{timestamp}.log", level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Script started.')

# Initialize a rich Console object
console = Console()

# IMPORTANT: if is_processing
is_processing_all = 1

# --- Main Loops ---
# Custom Progress display
with Progress(
    TextColumn("[progress.description]{task.description}"),
    BarColumn(),
    TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
    TimeRemainingColumn(),
    console=console
) as progress:
    
    # Outer loop over price range
    task_price = progress.add_task(
        "[green]Processing price ranges...", total=len(price_vec[:-1]-1)
    )

    for k, price in enumerate(price_vec[:-1]):
        params['pricefrom'] = round(price_vec[k])
        params['priceto'] = round(price_vec[k+1])

        if k % refresh_rate_cars_in_database == 0:
            response = supabase.table(table_name).select("car_id").execute()
            car_ids_in_database = {d['car_id'] for d in response.data}

        # Inner loop over mileage
        task_mileage = progress.add_task(
            f"[cyan]  Processing mileage {round(km_vec[0])}-{round(km_vec[-1])}...",
            total=len(km_vec[:-1]-1)
        )

        for j, km in enumerate(km_vec[:-1]):
            params['kmfrom'] = round(km_vec[j])
            params['kmto'] = round(km_vec[j+1])

            # Flag to check if the page loop completes fully
            page_limit_reached = True

            # Innermost loop over pages
            for i in range(page_limit_autoscout):
                params['page'] = i + 1
                html = requests.get(base_url, params=params).text
                soup = BeautifulSoup(html, "html.parser")
                car_listings = soup.find_all("article", class_="cldt-summary-full-item")

                if not car_listings:
                    # If no listings are found, the loop breaks early.
                    page_limit_reached = False
                    break

                for car in car_listings:
                    
                    # Check if car is already in database
                    car_id = car.get("id")
                    if car_id not in car_ids_in_upsert and ((car_id not in car_ids_in_database) or is_processing_all):

                        # Extract correct mileage
                        try:
                            data_mileage = float(car.get("data-mileage"))
                        except (ValueError, TypeError):
                            data_mileage = -1

                        # Extract correct listing price
                        try:
                            listing_price = float(car.get("data-price"))
                        except (ValueError, TypeError):
                            listing_price = -1

                        # Extract postcode
                        raw_postcode = car.get("data-listing-zip-code")
                        try:
                            postcode = raw_postcode[0:4] + raw_postcode[-2:].upper()
                            if not is_valid_format(postcode, pattern):
                                postcode = None
                        except:
                            postcode = None

                        # Extract the desired details by their data-testid attributes
                        transmission = car.find("span", {"data-testid": "VehicleDetails-transmission"})
                        fuel = car.find("span", {"data-testid": "VehicleDetails-gas_pump"})
                        power = car.find("span", {"data-testid": "VehicleDetails-speedometer"})

                        # Get the text values, stripping whitespace
                        transmission_text = transmission.get_text(strip=True) if transmission else None
                        fuel_text = fuel.get_text(strip=True) if fuel else None
                        power_text = power.get_text(strip=True) if power else None

                        kw_value = None
                        pk_value = None

                        if power_text:
                            # Extract numbers: first one before 'kW', second inside parentheses
                            match = re.search(r"(\d+)\s*kW.*\((\d+)\s*PK\)", power_text)
                            if match:
                                kw_value = float(match.group(1))
                                pk_value = float(match.group(2))

                        # Find car and model specifics
                        title_element = car.find("span", class_="ListItem_title_bold__iQJRq")
                        model_text = title_element.get_text(strip=True) if title_element else None
                        version_element = car.find("span", class_="ListItem_version__5EWfi")
                        version_text = version_element.get_text(strip=True) if version_element else None

                        # Find the actieradius span by aria-label
                        actieradius_element = car.find("span", attrs={"aria-label": "actieradius"})
                        actieradius_text = actieradius_element.get_text(strip=True) if actieradius_element else None

                        # Extract both numeric values as floats
                        ranges = [float(num) for num in
                                  re.findall(r"\d+(?:\.\d+)?", actieradius_text)] if actieradius_text else []

                        general_range = ranges[0] if len(ranges) > 0 else None
                        urban_range = ranges[1] if len(ranges) > 1 else None

                        car_info = {
                            "car_id": car_id,
                            "make": car.get("data-make"),
                            "model": car.get("data-model"),
                            "first_registration": car.get("data-first-registration"),
                            "fuel_type": car.get("data-fuel-type"),
                            "mileage": data_mileage,
                            "post_code_raw": raw_postcode,
                            "post_code": postcode,
                            "listing_price": listing_price,
                            "transmission": transmission_text,
                            "fuel_text": fuel_text,
                            "power_text": power_text,
                            "power_kw": kw_value,
                            "power_pk": pk_value,
                            "model_text": model_text,
                            "model_type": version_text,
                            "range_raw": actieradius_text,
                            "range_general": general_range,
                            "range_urban": urban_range
                        }
                        cars_to_insert.append(car_info)
                        car_ids_in_database.add(car_id)
                        car_ids_in_upsert.add(car_id)

                        # Update database in batches
                        if len(cars_to_insert) >= batch_size:
                            console.log(f"Inserting {len(cars_to_insert)} cars to the database...")
                            logging.info(f"Inserting {len(cars_to_insert)} cars to the database...")
                            supabase.table(table_name).upsert(cars_to_insert, ignore_duplicates=True).execute()
                            count_added += len(cars_to_insert)
                            cars_to_insert = []
                            car_ids_in_upsert = set()

                time.sleep(0.01)

            # Check and log if the page limit was reached for this mileage-price combination
            if page_limit_reached:
                console.log(f"Reached page limit for price: {params['pricefrom']}-{params['priceto']} and mileage: {params['kmfrom']}-{params['kmto']}")
                logging.info(f"Reached page limit for price: {params['pricefrom']}-{params['priceto']} and mileage: {params['kmfrom']}-{params['kmto']}")
                
            # Update the mileage task for each mileage range
            progress.update(task_mileage, advance=1)
        
        # Mark the mileage task as complete and remove it
        progress.remove_task(task_mileage)

        # Update the price task for each price range
        progress.update(task_price, advance=1)

    # Mark the price task as complete and remove it
    # progress.remove_task(task_price)

# --- Final Batch Insert ---
# Insert any remaining cars after all loops have finished
if cars_to_insert:
    console.log(f"Inserting final {len(cars_to_insert)} cars to the database...")
    supabase.table(table_name).upsert(cars_to_insert, ignore_duplicates=True).execute()
    count_added += len(cars_to_insert)

# console.log(f"\nTotal cars added to the database: {count_added}")
logging.info(f"\nTotal cars added to the database: {count_added}")
logging.info('Script finished successfully.')

Output()

[11:03:53] Inserting 502 cars to the database...                                                  ]8;id=568256;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=105653;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:08:42] Inserting 500 cars to the database...                                                  ]8;id=175296;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=980357;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:10:30] Inserting 500 cars to the database...                                                  ]8;id=63992;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=491864;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:13:42] Inserting 500 cars to the database...                                                  ]8;id=843149;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=817497;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

...

[11:32:12] Inserting 500 cars to the database...                                                  ]8;id=965975;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=551188;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:32:33] Inserting 500 cars to the database...                                                  ]8;id=49721;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=269487;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:32:54] Inserting 500 cars to the database...                                                  ]8;id=987220;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=779026;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:33:17] Inserting 500 cars to the database...                                                  ]8;id=969100;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=681517;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

[11:34:19] Inserting 500 cars to the database...                                                  ]8;id=592996;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py\2193470145.py]8;;\:]8;id=432000;file://C:\Users\david\AppData\Local\Temp\ipykernel_3880\2193470145.py#157\157]8;;\

In [99]:
cars_to_insert

[{'car_id': 'b83278ee-bf4e-409d-b196-a54031118d91',
  'make': 'peugeot',
  'model': '2008',
  'first_registration': '01-2025',
  'fuel_type': '2',
  'mileage': -1,
  'post_code_raw': '2182 DZ',
  'post_code': None,
  'listing_price': 429.0,
  'transmission': 'Automatisch',
  'fuel_text': 'Elektro/Benzine',
  'power_text': '108 kW (147 PK)',
  'power_kw': 108.0,
  'power_pk': 147.0,
  'model_text': 'Peugeot 2008',
  'model_type': '1.2 Hybrid 145 Style Super scherpe Private Lease a',
  'range_raw': None,
  'range_general': None,
  'range_urban': None},
 {'car_id': 'b83278ee-bf4e-409d-b196-a54031118d91',
  'make': 'peugeot',
  'model': '2008',
  'first_registration': '01-2025',
  'fuel_type': '2',
  'mileage': -1,
  'post_code_raw': '2182 DZ',
  'post_code': None,
  'listing_price': 429.0,
  'transmission': 'Automatisch',
  'fuel_text': 'Elektro/Benzine',
  'power_text': '108 kW (147 PK)',
  'power_kw': 108.0,
  'power_pk': 147.0,
  'model_text': 'Peugeot 2008',
  'model_type': '1.2 Hybri

In [100]:
car_ids__ = [d['car_id'] for d in cars_to_insert]

In [102]:
car_ids__

['b83278ee-bf4e-409d-b196-a54031118d91',
 'b83278ee-bf4e-409d-b196-a54031118d91',
 '99018bd3-90dd-4364-81a4-c0d466a10982',
 'd1e78be8-fda0-4295-b30d-cbd5ea62b061',
 '491d36ff-4e79-404b-b37b-361a6032d9d3',
 'f69b3dbe-d9b8-43a2-b87a-af6a5f0107e0',
 '7fd664f3-c601-4af7-b07f-fe5d4edc7e27',
 'e11833cd-1eca-472d-b6aa-ae2408ea2f63',
 '848558a2-3863-44a2-a8ac-9be019f8ad95',
 '8d6f7363-500a-4541-a40c-aae3b69281a6',
 '1c8d1151-6a89-4f47-8c9d-3b53fcbf65a6',
 '83c5f616-87b3-47f7-9593-c3200563d9f6',
 'f2e869c6-e5d2-4ae7-b5e0-f9e229221be9',
 '657bbb85-da4a-490e-8ad7-4c66bbbd0f8f',
 '1f751005-2c17-44a1-acbb-918f243397b5',
 '112cc42f-974a-4e61-926d-32a2c2d059e8',
 '0ae9ee3e-2064-467f-a3ea-785954f2fc53',
 '16612c25-e20a-480f-a302-5d5fb4bd32e0',
 '147f55e0-535f-4de3-8f7b-5b0d6ed9f9ea',
 'e14c6f92-560c-47c7-ad9e-03c845acaad3',
 'bb3d810a-aca0-4b99-94d0-503f1215cb8c',
 '9f04f51d-fad0-4083-9627-0ed3b529c391',
 'def2d589-364c-4347-aa17-15f9a5a4ae0f',
 '0e253cc9-e1aa-4fac-ac2c-3988a06b18c6',
 'bca650b7-8d64-

In [104]:
len(car_ids__)

501

In [ ]:
count_added

In [ ]:
response = supabase.table(table_name).select("car_id").execute()
car_ids_in_database = response.data

In [ ]:
df = pd.DataFrame([d['car_id'] for d in car_ids_in_database])

In [ ]:
len(df[0].unique()) - len(df)

## Remove duplicates in database

In [45]:
response = supabase.table(table_name).select("id, car_id, make, listing_price").execute()
car_ids_in_database = response.data

In [46]:
df_full = pd.DataFrame(car_ids_in_database)

In [47]:
df_full

,id,car_id,make,listing_price
0,77546,68936f70-81e9-4387-ad5c-ac6eed1f5438,nissan,28695
1,151,0d92d6f1-39e5-4606-b239-1eafa1eb65f2,fiat,9500
2,152,e148c182-bfdc-4820-befa-b7f8bb3569fe,mercedes-benz,20510
3,153,ac97cde6-8a38-4dca-a30a-bfb2286cb9ea,peugeot,19400
4,154,99c8af8d-6633-46c2-81e3-012d73c34c8b,mercedes-benz,27158
...,...,...,...,...
269541,276040,acf0832f-293c-4ac5-a276-fbf410baa346,fiat,3999
269542,276041,761a9158-01b3-4044-95ee-7e5ae70b40ad,kia,3699
269543,276042,cec055f4-e88a-4965-9238-a226e6313057,mini,3800
269544,276043,8bc1bca9-2ba0-4294-b80b-3a919effa566,mercedes-benz,3900


In [48]:
id_to_remove = df_full.loc[(df_full.duplicated(subset=['car_id'],keep="first")), 'id'].values

In [49]:
id_to_remove

array([398112, 398113, 398128, 398131, 398132, 398133, 398134, 398135,
       398136, 398137, 398138, 398139, 398140, 398141, 398142, 398143,
       398144, 398145, 398146, 398147, 398148, 398149, 398150, 398151,
       398152, 398153, 398154, 398155, 398156, 398157, 398158, 398159,
       398160, 398161, 398162, 398163, 398164, 398165, 398166, 398167,
       398168, 398169, 398170, 398171, 398172, 398173, 398174, 398175,
       398177, 398178, 398179, 398180, 398181, 398182, 398183, 398184,
       398185, 398186, 398187, 398188, 398189, 398190, 398191, 398192,
       398193, 398194, 398195, 398196, 398197, 398198, 398199, 398200,
       398201, 398202, 398203, 398204, 398205, 398206, 398207, 398208,
       398209, 398210, 398211, 398212, 398213, 398214, 398215, 398216,
       398217, 398218, 398219, 398220, 398221, 398222, 398223, 398224,
       398225, 398226, 398227, 398228, 398229, 398230, 398232, 398233,
       398234, 398235, 398236, 398237, 398238, 398239, 398240, 398241,
      

In [50]:
chunk_size = 1000

for i in tqdm(range(0, len(id_to_remove), chunk_size)):
    chunk = id_to_remove[i:min(i + chunk_size, len(id_to_remove))]
    response = (
        supabase.table(table_name)
        .delete()
        .in_("id", chunk)
        .execute()
    )


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


## Check latitude and longitude

In [84]:
BASE_URL = "https://openpostcode.nl/api/address"

params = {
    "postcode": "1011AB",
    "huisnummer": '99'
}

response = requests.get(BASE_URL, params=params)

In [85]:
response.json()

{'postcode': '1011AB',
 'huisnummer': '99',
 'straat': 'De Ruijterkade',
 'buurt': 'Oosterdokseiland',
 'wijk': 'Nieuwmarkt/Lastage',
 'woonplaats': 'Amsterdam',
 'gemeente': 'Amsterdam',
 'provincie': 'Noord-Holland',
 'latitude': 52.378666,
 'longitude': 4.9054437}

## Check post code convention

In [184]:
def is_valid_format(s, pattern):
    return bool(re.fullmatch(pattern, s))

In [187]:
response = supabase.table(table_name).select("id, car_id, post_code, post_code_raw").execute()

In [188]:
df_full = pd.DataFrame(response.data)

In [189]:
df_full['post_code'].sort_values().unique()

array(['0000AA', '0000AB', '0000OO', ..., '9997PG', '9998NZ', None],
      shape=(31130,), dtype=object)

In [190]:
df_full

,id,car_id,post_code,post_code_raw
0,77546,68936f70-81e9-4387-ad5c-ac6eed1f5438,8281PC,8281 PC
1,151,0d92d6f1-39e5-4606-b239-1eafa1eb65f2,2987VD,2987 VD
2,152,e148c182-bfdc-4820-befa-b7f8bb3569fe,7903BM,7903 BM
3,153,ac97cde6-8a38-4dca-a30a-bfb2286cb9ea,7513EL,7513 EL
4,154,99c8af8d-6633-46c2-81e3-012d73c34c8b,5505JA,5505 JA
...,...,...,...,...
268541,276040,acf0832f-293c-4ac5-a276-fbf410baa346,5161CB,5161 CB
268542,276041,761a9158-01b3-4044-95ee-7e5ae70b40ad,2988AD,2988 AD
268543,276042,cec055f4-e88a-4965-9238-a226e6313057,7943KC,7943KC
268544,276043,8bc1bca9-2ba0-4294-b80b-3a919effa566,9482RD,9482rd


In [166]:
batch_size = 100  # choose a batch size to avoid huge payloads
updates = []

for index, row in tqdm(df_full.iterrows(), total=df_full.shape[0]):
    if row['post_code']:  # skip if already set
        continue
    
    raw_postcode = row['post_code_raw']
    
    # Compute formatted postcode
    try:
        postcode = raw_postcode[0:4] + raw_postcode[-2:].upper()
        if not is_valid_format(postcode, pattern):
            postcode = None
    except Exception as e:
        postcode = None
    
    # Append to batch list
    updates.append({"id": row['id'], "post_code": postcode})
    
    # When batch is full, send to Supabase
    if len(updates) >= batch_size:
        supabase.table(table_name).upsert(updates).execute()
        updates = []  # clear batch

# Send remaining updates if any
if updates:
    supabase.table(table_name).upsert(updates).execute()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268546/268546 [05:58<00:00, 749.33it/s]


In [182]:
BASE_URL = "https://openpostcode.nl/api/address"

params = {
    "postcode": "4167BM",
    "huisnummer": '1'
}

response = requests.get(BASE_URL, params=params)

In [183]:
response.text

'{"error":"Postcode not found"}\n'